<a href="https://colab.research.google.com/github/BlackArctic20/BluepringComparisonIA/blob/main/Eduardo1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install pdf2image


In [ ]:
!apt-get install -y poppler-utils

In [21]:
# -*- coding: utf-8 -*-
"""
Created on Tue Mar 25 10:59:43 2025

@author: HUJ1GA
"""

import tensorflow as tf
import numpy as np
from pdf2image import convert_from_path
import cv2
from PIL import Image
import os

# Function to convert the first page of a PDF to a PNG image
def pdf_to_image(pdf_path, output_path):
    """Converts the first page of a PDF to a PNG image."""
    try:
        pages = convert_from_path(pdf_path, dpi=200)  # Convert PDF to list of PIL Image objects (one per page)
    except Exception as e:
        print(f"Error converting PDF: {e}")  # Handle potential conversion errors
        return None

    if not os.path.exists(output_path):  # Create output directory if it doesn't exist
        os.makedirs(output_path)

    if pages:  # Check if any pages were converted
        image_path = os.path.join(output_path, 'page.png')
        pages[0].save(image_path, 'PNG')  # Save the first page as PNG
        return image_path
    else:
        print("No pages converted from PDF.")
        return None



# Function to preprocess image for TensorFlow
def preprocess_image(image_path):
    """Preprocesses an image for TensorFlow."""
    if not image_path: # Check if image_path is valid
        return None

    img = cv2.imread(image_path)
    if img is None: #Check if image is loaded correctly
        print("Error loading image.")
        return None
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert from BGR (OpenCV default) to RGB
    img = cv2.resize(img, (512, 512))  # Resize to a consistent size
    img_tensor = tf.convert_to_tensor(img, dtype=tf.float32) / 255.0  # Convert to tensor and normalize to [0, 1]
    return img_tensor



# Function to compare images and highlight differences
def compare_images(old_img_tensor, new_img_tensor, output_path):
    """Compares two images and highlights differences in red."""
    if old_img_tensor is None or new_img_tensor is None:
        print("Cannot compare. One or both images are invalid")
        return

    # Compute absolute difference
    diff = tf.abs(old_img_tensor - new_img_tensor)

    # Threshold to highlight significant differences
    threshold = 0.1  # Adjust this value as needed
    diff_binary = tf.where(diff > threshold, 1.0, 0.0)

    # Convert back to image format (NumPy array)
    diff_binary_np = diff_binary.numpy()

    # Create a colored overlay (RGB image)
    overlay = np.zeros_like(old_img_tensor.numpy())  # Initialize overlay with zeros

    # Apply red color where difference is above threshold
    red_indices = diff_binary_np > 0
    overlay[red_indices] = [255, 0, 0]  # Set pixels to red (RGB: 255, 0, 0)

    # Combine original image with overlay, using appropriate data type
    blended_image = old_img_tensor.numpy() * 255 +  overlay
    blended_image = np.clip(blended_image, 0, 255).astype(np.uint8)


    # Save the result
    cv2.imwrite(output_path, cv2.cvtColor(blended_image, cv2.COLOR_RGB2BGR)) # convert back to BGR
    print(f"Differences saved to {output_path}")



# Main execution
def main():
    """Compares two PDF drawings and saves the differences image."""
    old_pdf_path = "00_Honda.pdf"  # Replace with the path to your old PDF
    new_pdf_path = "01_Honda.pdf"  # Replace with the path to your new PDF

    output_dir = "output"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    result_path = os.path.join(output_dir, "differences.png")


    # Convert PDFs to images
    old_image_path = pdf_to_image(old_pdf_path, os.path.join(output_dir, "old"))
    new_image_path = pdf_to_image(new_pdf_path, os.path.join(output_dir, "new"))


    # Preprocess images (convert and resize to tensors)
    old_img_tensor = preprocess_image(old_image_path)
    new_img_tensor = preprocess_image(new_image_path)


    # Compare and save differences
    compare_images(old_img_tensor, new_img_tensor, result_path)



if __name__ == "__main__":
    main()

ValueError: NumPy boolean array indexing assignment cannot assign 3 input values to the 92271 output values where the mask is true